# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
#importing the dataset

orders = pd.read_csv('/Users/deshdeepak/lab-dataframe-calculations/your-code/Orders.csv')
orders.head(5)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [11]:
#subproblem1: aggregating the amount_spent for unique customers

# To make run all you ask in one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

print(orders.columns)

#unnamed looks like a copy of index from old version, dropping ot
#orders = orders.drop(['Unnamed: 0'],axis=1) -- already dropped, # to it so it doesn't run again and gives error
orders.head()

Index(['Unnamed: 0', 'InvoiceNo', 'StockCode', 'year', 'month', 'day', 'hour',
       'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID',
       'Country', 'amount_spent'],
      dtype='object')


,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [33]:
#we have months, days and hours, and amount_spent, columns so we can aggregate amount_spent in our preferred way. I am going to aggregate them by months

#creating a pivot table and running groupby to see if we can get customer_id and amount_spent

grouped_orders = orders['amount_spent'].groupby(by=orders['CustomerID'],axis=0, dropna=True).sum()
grouped_orders = pd.DataFrame(grouped_orders)
grouped_orders = grouped_orders.reset_index()
grouped_orders.head(5)

grouped_orders.describe()
pivot = orders.pivot_table(index=['month','CustomerID'],values=['amount_spent'],dropna=True)
pivot

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


,CustomerID,amount_spent
count,4339.000000,4339.000000
mean,15299.936852,2053.793018
std,1721.889758,8988.248381
min,12346.000000,0.000000
25%,13812.500000,307.245000
50%,15299.000000,674.450000
75%,16778.500000,1661.640000
max,18287.000000,280206.020000


amount_spent
month CustomerID              
1     12346       77183.600000
      12347          16.392759
      12348          37.906667
      12356          63.100556
      12359          34.218750
...                        ...
12    18269          24.085714
      18272          15.994783
      18273          51.000000
      18282          15.568000
      18283           4.160000

[12820 rows x 1 columns]

Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?

we just need to filter out customers who are preferred 75 percentile and/or VIP 95 as per the question posed to us. but we could do that for any percentile range

perhaps we don't to have overlap of preferred and VIP customers, so preferred customers will be between 75 and 95 percentile.

We could then filter customers in any given quantile range.


In [56]:
#Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

#applying deduction from sub problem 2 concept to define a function to label these customers in a new column
grouped_orders
def quantile_labels(rows):
    if np.percentile(rows['amount_spent'],95)>=np.percentile(grouped_orders['amount_spent'],95):
        return 'VIP'
    elif np.percentile(rows['amount_spent'],75)<=np.percentile(grouped_orders['amount_spent'],75):
        return np.nan
    else:
        return 'Preferred'
    
grouped_orders['customer_labeled'] = grouped_orders.apply(quantile_labels, axis = 1)
grouped_orders = grouped_orders.dropna()
grouped_orders


,CustomerID,amount_spent,customer_labeled,filter
0,12346,77183.60,VIP,True
1,12347,4310.00,Preferred,True
2,12348,1797.24,Preferred,True
3,12349,1757.55,Preferred,True
5,12352,2506.04,Preferred,True
...,...,...,...,...
4319,18259,2338.60,Preferred,True
4320,18260,2643.20,Preferred,True
4328,18272,3078.58,Preferred,True
4337,18283,2094.88,Preferred,True


<ipython-input-56-b53a5426d5d0>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped_orders['customer_labeled'] = grouped_orders.apply(quantile_labels, axis = 1)


,CustomerID,amount_spent,customer_labeled,filter
0,12346,77183.60,VIP,True
10,12357,6207.67,Preferred,True
12,12359,6372.58,Preferred,True
15,12362,5226.23,Preferred,True
50,12409,11072.67,Preferred,True
...,...,...,...,...
4273,18198,5425.56,Preferred,True
4292,18223,6484.54,Preferred,True
4294,18225,5509.12,Preferred,True
4295,18226,5228.05,Preferred,True


In [57]:
grouped_orders['filter'] = ((grouped_orders['customer_labeled']=='VIP')|(grouped_orders['customer_labeled']=='Preferred'))
#
#
#grouped_orders['filter'].drop('False',axis=1)
grouped_orders.head(5)

<ipython-input-57-4f69ad728f03>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped_orders['filter'] = ((grouped_orders['customer_labeled']=='VIP')|(grouped_orders['customer_labeled']=='Preferred'))


,CustomerID,amount_spent,customer_labeled,filter
0,12346,77183.60,VIP,True
10,12357,6207.67,Preferred,True
12,12359,6372.58,Preferred,True
15,12362,5226.23,Preferred,True
50,12409,11072.67,Preferred,True


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [64]:
# adding countries to this df
country_vip = pd.DataFrame(orders['Country'],orders['CustomerID'])
country_vip = country_vip.reset_index()
country_vip = country_vip.drop_duplicates('CustomerID')

country_vip.head(10)
                     

,CustomerID,Country
0,17850,United Kingdom
9,13047,United Kingdom
26,12583,United Kingdom
46,13748,United Kingdom
65,15100,United Kingdom
82,15291,EIRE
86,14688,United Kingdom
105,17809,United Kingdom
106,15311,EIRE
141,16098,United Kingdom


In [65]:
#merging this df to my grouped_orders df

merged_c_g = grouped_orders.merge(country_vip, left_on='CustomerID', right_on='CustomerID')
merged_c_g.sample(25)

,CustomerID,amount_spent,customer_labeled,filter,Country
32,12683,8476.57,Preferred,True,Norway
53,13004,5655.91,Preferred,True,United Kingdom
114,14367,9261.52,Preferred,True,United Kingdom
268,18225,5509.12,Preferred,True,United Kingdom
166,15640,12515.66,Preferred,True,United Kingdom
36,12731,18895.91,VIP,True,United Kingdom
235,17416,6893.42,Preferred,True,United Kingdom
202,16652,6773.97,Preferred,True,United Kingdom
191,16303,5360.63,Preferred,True,United Kingdom
170,15749,44534.30,VIP,True,United Kingdom


In [67]:
merged_c_g.dtypes #CustomerID should be converted into object

merged_c_g['CustomerID']=merged_c_g['CustomerID'].astype(object)
merged_c_g.dtypes

CustomerID            int64
amount_spent        float64
customer_labeled     object
filter                 bool
Country              object
dtype: object

CustomerID           object
amount_spent        float64
customer_labeled     object
filter                 bool
Country              object
dtype: object

In [69]:
#filtering VIP customers-country

vip_country = merged_c_g[merged_c_g['customer_labeled']=='VIP']
vip_country.sort_values(by='Country').max()

CustomerID                   18102
amount_spent             280206.02
customer_labeled               VIP
filter                        True
Country             United Kingdom
dtype: object

The country with most VIP customers is United Kingdom!

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [70]:
#filtering VIP+preferred customers by country

vip_pref_country = merged_c_g[(merged_c_g['customer_labeled']=='VIP')|(merged_c_g['customer_labeled']=='Preferred')]
vip_pref_country.sort_values(by='Country').max()

CustomerID                   18229
amount_spent             280206.02
customer_labeled               VIP
filter                        True
Country             United Kingdom
dtype: object

The country with most VIP and Preferred customers is the United Kingdom!